In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("../")

import pykep as pk
import numpy as np
import matplotlib.pyplot as plt
import math
import os

starting_t = pk.epoch_from_string('2022-01-01 00:00:00.000')

# 1. Initialize Constellation and reset planet_list and other lists

In [ ]:
#---Constellation data----------------------------------------------------------------------------------------------
constellation_name = "AstraPhase3"
startTime = 0
duration = 0
#-------------------------------------------------------------------------------------------------------------------
nShells = 0
planet_list = []
shell_list = []
xparams_list = []
print("Planet list is now empty")

# 2.1 Generate shell = ( altitude, inclination, nPlanes, nSats )

In [ ]:
# creates a temporary shell that has to be appended to planet_list in cell "Append to planet_list and other"
# to be stored and not get overwritten (this also concerns additional shell data shell_list, xparams_list)

#---input-----------------------------------------------------------------------------------------------------------
altitude = 400
inclination = 55.0
nPlanes = 61
nSats = 68
#-------------------------------------------------------------------------------------------------------------------
#---extra-params----------------------------------------------------------------------------------------------------

#offsetM = offset for Mean Anomaly added after each plane (relative phasing)
# walker constellation: offsetM = F * 360 / (nPlanes * nSats) ; F element {0, ... , nPlanes - 1}
offsetM = 0    # default 0

#argPeriapsis = argument of periapsis
# starting point of satellite placement for each plane
# argPeriapsis = pi avoids collisions in planes that intersect at reference plane
argPeriapsis = math.pi  # default math.pi 

#startingW = offset for W that is not accumulating (W = longitude of ascending node)
# formula for overlapping shells (same altitude, same inclination):
# (360 / G) / 2 ; G = smallest common multiple of the overlapping nPlanes
startingW = 0  # default 0

# W_area: orbital planes are distributed evenly within range [startingW,startingW + maximumW)
W_area = 360     # default 360
#-------------------------------------------------------------------------------------------------------------------

minW = startingW
maxW = W_area + startingW

a = altitude * 1000 + 6371000    # in [m], earth radius included
e = 0
i = inclination * pk.DEG2RAD
W = pk.DEG2RAD * minW
w = argPeriapsis * pk.DEG2RAD       
M = 0             

plane_count = 0

when = starting_t
mu_central_body = pk.MU_EARTH
mu_self = 1
radius = 1
safe_radius = 1

pStep = pk.DEG2RAD * W_area / nPlanes  # W goes from startingW to startingW+W_area
sStep = 2 * math.pi / nSats              # M goes from 0° to 360°
sExtraStep = pk.DEG2RAD*offsetM

planet_list_tmp = []
for x in range(nPlanes):
    for y in range(nSats):
        planet_list_tmp.append(pk.planet.keplerian(when,[a,e,i,W,w,M],mu_central_body,mu_self,radius,safe_radius,"sat"))
        M = M + sStep
    plane_count = plane_count + 1
    W = W + pStep
    M = plane_count * sExtraStep     #equals 0 + count*0 = 0 in the usual case
    
shell = (altitude,inclination,nPlanes,nSats)
xparams = (offsetM,argPeriapsis,startingW,W_area)

nShells += 1

print("Added " + str(len(planet_list_tmp)) + " planets")

# 2.2 Append to planet_list and other

In [ ]:
# stores and appends temporary shell in planet_list (including lists with additional data)
planet_list = planet_list + planet_list_tmp
shell_list.append(shell)
xparams_list.append(xparams)
print("Added " + str(len(planet_list_tmp)) + " to planet_list now totalling " + str(len(planet_list)) + " planets.")

## Plot current shell

In [ ]:
fig = plt.figure(figsize=(6,6),dpi=100)
ax = plt.axes(projection='3d')
for i in range (nPlanes*nSats):
    pk.orbit_plots.plot_planet(planet_list_tmp[i],axes=ax,s=20)

# 3. Create position and velocity vectors

In [ ]:
objects = []
count_too_low = 0
count_too_high = 0
for planet in planet_list:
    try:
        pos,v = planet.eph(starting_t)
        
        # convert to km and numpy
        pos = np.asarray(pos) / 1000.0 
        v = np.asarray(v) / 1000.0
        
        objects.append((pos,v))
    except RuntimeError as e:
        print(e, " propagating ",planet.name)
        
print("Successfully propagated ",len(objects)," objects.")

# 4. Plot and store results

In [ ]:
fig = plt.figure(figsize=(6,6),dpi=100)
ax = plt.axes(projection='3d');
# pos , v (.csv)
positions = np.array([pos for pos,_ in objects])
velocities = np.array([v for _,v in objects])
ax.scatter(positions[:,0],positions[:,1],positions[:,2],".",alpha=0.25)
# parameters (.yaml)
constellation_yaml = ""
constellation_yaml += "constellation:\n"
constellation_yaml += "  name: " + constellation_name + "\n"
constellation_yaml += "  startTime: " + str(startTime) + "\n"
constellation_yaml += "  duration: " + str(duration) + "\n"
constellation_yaml += "  nShells: " + str(nShells) + "\n\n"
for i in range(nShells):
    constellation_yaml += "shell" + str(i+1) + ":\n"
    constellation_yaml += "  altitude: " + str(shell_list[i][0]) + "\n"
    constellation_yaml += "  inclination: " + str(shell_list[i][1]) + "\n"
    constellation_yaml += "  nPlanes: " + str(shell_list[i][2]) + "\n"
    constellation_yaml += "  nSats: " + str(shell_list[i][3]) + "\n\n"
for i in range(nShells):
    constellation_yaml += "special" + str(i+1) + ":\n"
    constellation_yaml += "  offsetM: " + str(xparams_list[i][0]) + "\n"
    constellation_yaml += "  argPeriapsis: " + str(xparams_list[i][1]) + "\n"
    constellation_yaml += "  startingW: " + str(xparams_list[i][2]) + "\n"
    constellation_yaml += "  W_area: " + str(xparams_list[i][3]) + "\n\n"

# 5. Save Constellation as files in directory

In [ ]:
# creates directory named as the constellation_name value within the data folder and stores output there

os.mkdir("../../data/" + constellation_name);

np.savetxt("../../data/" + constellation_name + "/pos_"+constellation_name+".csv",positions,delimiter=",")
np.savetxt("../../data/" + constellation_name + "/v_"+constellation_name+".csv",velocities,delimiter=",")

f1 = open("../../data/" + constellation_name + "/shells_"+constellation_name+".yaml","w")
f1.write(constellation_yaml)
f1.close()